In [1]:
include("import_all.jl")

In [2]:
# io = open("log.txt", "w+")
# logger = SimpleLogger(io)
logger = ConsoleLogger(stdout, Logging.Info)
# fileLogger = SimpleLogger(io, Logging.Debug)
# global_logger(fileLogger)
global_logger(logger)

ConsoleLogger(IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(Base.Libc.WindowsRawSocket(0x00000000000003cc) open, 0 bytes waiting))), Info, Logging.default_metafmt, true, 0, Dict{Any, Int64}())

In [3]:
using Profile
using PProf

In [4]:
function wrapper()
    max_time = 1440
    start_time = 345


    train_period = 2
    train_capacity = 1000

    @info "$(now()): initialization starting at time "
    station_dict = construct_station_dict("data/input/station_data.csv")

    station_name_id_map = construct_station_name_id_map(station_dict)

    # construct the edges
    start_stations = construct_edges_from_edges_dict!(station_dict, ["tel", "ccl", "ewl", "nsl", "nel", "cgl", "dtl"])

    lines = construct_lines_from_start_stations(station_dict, start_stations)

    commuter_graph = construct_commuter_graph(station_dict)

    floyd_warshall!(commuter_graph)

    get_all_path_pairs!(commuter_graph)

    paths = get_interchange_paths(station_dict, lines, commuter_graph)

    trains = Dict()
    event_queue = []
    for line_code in keys(lines)
        line_duration = get_line_duration(station_dict, lines, line_code)
        depot_id = lines[line_code]["FW"][1]
        result = create_period_train_placement_events(line_code, line_duration, train_period, train_capacity, depot_id, "FW", start_time)

        for (k,v) in result["trains"]
            trains[k] = v 
        end 

        append!(event_queue, result["events"])
    end

    spawn_events = create_spawn_events!("data/input/spawn_data.csv", station_dict, start_time)

    append!(event_queue, spawn_events)

    metro = Metro(station_dict, trains, lines, paths);

    build_min_heap!(event_queue)

    data_store = Data_Store(Dict(), Dict(), Dict(), Dict(), Dict())
    @info "$(now()): initialization finish "

    @info "$(now()): starting simulation "
    final_data = simulate!(max_time, metro, event_queue, data_store)
    @info "$(now()): ending simulation "
end


wrapper (generic function with 1 method)

In [5]:
wrapper()

[ Info: 2023-04-18T16:22:01.753: initialization starting at time 
[ Info: 2023-04-18T16:22:20.267: initialization finish 
[ Info: 2023-04-18T16:22:20.267: starting simulation 
[ Info: 2023-04-18T16:22:20.328: start time is 5
[ Info: processed 5272 events
[ Info: 2023-04-18T16:22:26.167: time is now 6
[ Info: size of event queue 17153
[ Info: processed 100468 events
[ Info: 2023-04-18T16:22:29.340: time is now 7
[ Info: size of event queue 17142
[ Info: processed 261358 events
[ Info: 2023-04-18T16:22:36.384: time is now 8
[ Info: size of event queue 17053
[ Info: processed 406645 events
[ Info: 2023-04-18T16:22:44.775: time is now 9
[ Info: size of event queue 16932
[ Info: processed 504803 events
[ Info: 2023-04-18T16:22:50.278: time is now 10
[ Info: size of event queue 16840
[ Info: processed 588441 events
[ Info: 2023-04-18T16:22:53.219: time is now 11
[ Info: size of event queue 16749
[ Info: processed 675222 events
[ Info: 2023-04-18T16:22:56.375: time is now 12
[ Info: size of e

In [6]:
using BenchmarkTools

In [7]:
@benchmark wrapper()

[ Info: 2023-04-18T16:23:40.596: initialization starting at time 
[ Info: 2023-04-18T16:23:47.140: initialization finish 
[ Info: 2023-04-18T16:23:47.140: starting simulation 
[ Info: 2023-04-18T16:23:47.140: start time is 5
[ Info: processed 5247 events
[ Info: 2023-04-18T16:23:47.308: time is now 6
[ Info: size of event queue 17150
[ Info: processed 100392 events
[ Info: 2023-04-18T16:23:49.750: time is now 7
[ Info: size of event queue 17124
[ Info: processed 261445 events
[ Info: 2023-04-18T16:23:54.781: time is now 8
[ Info: size of event queue 17037
[ Info: processed 406027 events
[ Info: 2023-04-18T16:23:59.975: time is now 9
[ Info: size of event queue 16938
[ Info: processed 503293 events
[ Info: 2023-04-18T16:24:03.680: time is now 10
[ Info: size of event queue 16835
[ Info: processed 586944 events
[ Info: 2023-04-18T16:24:06.594: time is now 11
[ Info: size of event queue 16724
[ Info: processed 674055 events
[ Info: 2023-04-18T16:24:09.338: time is now 12
[ Info: size of e

[ Info: size of event queue 13473
[ Info: processed 1675810 events
[ Info: 2023-04-18T16:25:34.873: time is now 20
[ Info: size of event queue 12033
[ Info: processed 1769998 events
[ Info: 2023-04-18T16:25:37.071: time is now 21
[ Info: size of event queue 10591
[ Info: processed 1857018 events
[ Info: 2023-04-18T16:25:39.131: time is now 22
[ Info: size of event queue 8337
[ Info: processed 1923035 events
[ Info: 2023-04-18T16:25:40.806: time is now 23
[ Info: size of event queue 4798
[ Info: processed 1962455 events
[ Info: 2023-04-18T16:25:41.902: time is now 24
[ Info: size of event queue 582
[ Info: 2023-04-18T16:25:41.925: ending simulation 
[ Info: storing data
[ Info: 2023-04-18T16:25:44.382: initialization starting at time 
[ Info: 2023-04-18T16:25:49.044: initialization finish 
[ Info: 2023-04-18T16:25:49.068: starting simulation 
[ Info: 2023-04-18T16:25:49.068: start time is 5
[ Info: processed 5199 events
[ Info: 2023-04-18T16:25:49.168: time is now 6
[ Info: size of even

[ Info: size of event queue 16485
[ Info: processed 867133 events
[ Info: 2023-04-18T16:27:14.322: time is now 14
[ Info: size of event queue 16326
[ Info: processed 965324 events
[ Info: 2023-04-18T16:27:16.666: time is now 15
[ Info: size of event queue 16155
[ Info: processed 1069986 events
[ Info: 2023-04-18T16:27:19.070: time is now 16
[ Info: size of event queue 15945
[ Info: processed 1185875 events
[ Info: 2023-04-18T16:27:21.773: time is now 17
[ Info: size of event queue 15652
[ Info: processed 1356268 events
[ Info: 2023-04-18T16:27:25.332: time is now 18
[ Info: size of event queue 14963
[ Info: processed 1552221 events
[ Info: 2023-04-18T16:27:29.540: time is now 19
[ Info: size of event queue 13475
[ Info: processed 1675832 events
[ Info: 2023-04-18T16:27:32.484: time is now 20
[ Info: size of event queue 12079
[ Info: processed 1770297 events
[ Info: 2023-04-18T16:27:34.775: time is now 21
[ Info: size of event queue 10665
[ Info: processed 1857139 events
[ Info: 2023-04

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 55.833 s (7.39% GC) to evaluate,
 with a memory estimate of 26.34 GiB, over 587174429 allocations.

In [8]:
@profile wrapper()

[ Info: 2023-04-14T09:42:06.722: initialization starting at time 
[ Info: 2023-04-14T09:42:14.517: initialization finish 
[ Info: 2023-04-14T09:42:14.517: starting simulation 
[ Info: 2023-04-14T09:42:14.518: start time is 5
[ Info: processed 5200 events
[ Info: 2023-04-14T09:42:14.662: time is now 6
[ Info: size of event queue 17121
[ Info: processed 100686 events
[ Info: 2023-04-14T09:42:17.911: time is now 7
[ Info: size of event queue 17117
[ Info: processed 261697 events
[ Info: 2023-04-14T09:42:24.144: time is now 8
[ Info: size of event queue 17026
[ Info: processed 406265 events
[ Info: 2023-04-14T09:42:30.308: time is now 9
[ Info: size of event queue 16909
[ Info: processed 504143 events
[ Info: 2023-04-14T09:42:36.221: time is now 10
[ Info: size of event queue 16809
[ Info: processed 587787 events
[ Info: 2023-04-14T09:42:42.525: time is now 11
[ Info: size of event queue 16707
[ Info: processed 674529 events
[ Info: 2023-04-14T09:42:49.620: time is now 12
[ Info: size of e

In [9]:
pprof(;webport=58698)

"profile.pb.gz"

In [10]:
Profile.print(format=:tree, recur=:flat)

Overhead ╎ [+additional indent] Count File:Line; Function
   ╎12325 @Base\task.jl:484; (::IJulia.var"#15#18")()
   ╎ 12325 ...lia\src\eventloop.jl:8; eventloop(socket::ZMQ.Socket)
   ╎  12325 @Base\essentials.jl:726; invokelatest
   ╎   12325 @Base\essentials.jl:729; #invokelatest#2
   ╎    12325 ...execute_request.jl:67; execute_request(socket::ZMQ.S...
   ╎     12325 ...SoftGlobalScope.jl:65; softscope_include_string(m::...
   ╎    ╎ 12325 @Base\loading.jl:1428; include_string(mapexpr::typ...
   ╎    ╎  12325 @Base\boot.jl:368; eval
   ╎    ╎   1     In[6]:21; wrapper()
   ╎    ╎    1     ...ion_functions.jl:51; construct_edges_from_edges...
   ╎    ╎     1     ...ion_functions.jl:43; create_station_code_map(s...
   ╎    ╎    ╎ 1     @Base\dict.jl:398; setindex!(h::Dict{Any, A...
   ╎    ╎    ╎  1     @Base\dict.jl:327; ht_keyindex2!(h::Dict{An...
   ╎    ╎    ╎   1     @Base\array.jl:924; getindex
   ╎    ╎   297   In[6]:27; wrapper()
 30╎    ╎    71    ...ing_functions.jl:0; floyd_

Main binary filename not available.
Serving web UI on http://localhost:58698



   ╎    ╎    ╎    ╎  1     ...r\inlining.jl:82; ssa_inlining_pass!(ir...
   ╎    ╎    ╎    ╎   1     ...\inlining.jl:1425; assemble_inline_tod...
   ╎    ╎    ╎    ╎    1     ...inlining.jl:1210; analyze_single_call...
   ╎    ╎    ╎    ╎     1     ...inlining.jl:1293; handle_match!(matc...
   ╎    ╎    ╎    ╎    ╎ 1     ...inlining.jl:861; analyze_method!(ma...
   ╎    ╎    ╎    ╎    ╎  1     ...inlining.jl:804; resolve_todo(todo:...
  1╎    ╎    ╎    ╎    ╎   1     ...nlining.jl:870; InliningTodo
   ╎    ╎    ╎    1     ...r\typeinfer.jl:279; _typeinf(interp::Core....
   ╎    ╎    ╎     1     ...\typeinfer.jl:223; finish!
   ╎    ╎    ╎    ╎ 1     ...r\optimize.jl:145; ir_to_codeinf!
   ╎    ╎    ╎    ╎  1     ...air\legacy.jl:42; replace_code_newstyle...
   ╎    ╎    ╎    ╎   1     @Base\array.jl:1241; resize!
  1╎    ╎    ╎    ╎    1     @Base\array.jl:1020; _deleteend!
  1╎    ╎     1     @Base\dict.jl:496; getindex(h::Dict{String, ...
   ╎    ╎     22    @Base\dict.jl:497; getin

   ╎    ╎    14    ...ter_functions.jl:7; create_spawn_events!(spawn...
   ╎    ╎     14    @CSV\src\file.jl:162; Type##kw
   ╎    ╎    ╎ 14    @CSV\src\file.jl:222; #File#28
   ╎    ╎    ╎  14    @CSV\src\file.jl:226; File
   ╎    ╎    ╎   12    @CSV\src\file.jl:289; CSV.File(ctx::CSV.Conte...
   ╎    ╎    ╎    12    @CSV\src\file.jl:549; parsefilechunk!(ctx::C...
   ╎    ╎    ╎     1     @CSV\src\file.jl:604; parserow
   ╎    ╎    ╎    ╎ 1     @CSV\src\file.jl:766; parsevalue!(#unused#:...
   ╎    ╎    ╎    ╎  1     ...rc\Parsers.jl:348; xparse
   ╎    ╎    ╎    ╎   1     ...c\Parsers.jl:350; xparse(::Type{Int64}...
   ╎    ╎    ╎    ╎    1     ...c\Parsers.jl:342; _xparse
   ╎    ╎    ╎    ╎     1     ...omponents.jl:9; #6
   ╎    ╎    ╎    ╎    ╎ 1     ...mponents.jl:36; checkemptysentinel
   ╎    ╎    ╎    ╎    ╎  1     ...mponents.jl:364; _finddelimiter
   ╎    ╎    ╎    ╎    ╎   1     ...mponents.jl:85; stripwhitespace
   ╎    ╎    ╎    ╎    ╎    1     ...ponents.jl:216; findquo

   ╎    ╎    ╎     1     ...s\substring.jl:39; SubString
   ╎    ╎    ╎    ╎ 1     ...\substring.jl:38; SubString
   ╎    ╎    ╎    ╎  1     ...\substring.jl:34; SubString
  1╎    ╎    ╎    ╎   1     ...gs\string.jl:154; nextind
  1╎    ╎    2     ...ter_functions.jl:28; create_spawn_events!(spawn...
  1╎    ╎     1     @Base\array.jl:898; iterate(A::Vector{String})
  1╎    ╎    1     ...ter_functions.jl:29; create_spawn_events!(spawn...
  2╎    ╎    3     ...ter_functions.jl:30; create_spawn_events!(spawn...
   ╎    ╎     1     @Base\dict.jl:498; getindex(h::Dict{Any, Any...
  1╎    ╎    ╎ 1     @Base\array.jl:924; getindex
  5╎    ╎    12    ...ter_functions.jl:35; create_spawn_events!(spawn...
   ╎    ╎     7     ...s\src\Tables.jl:184; getindex(r::CSV.Row, i::I...
  7╎    ╎    ╎ 7     @CSV\src\file.jl:30; getcolumn
  2╎    ╎    2     ...ter_functions.jl:37; create_spawn_events!(spawn...
  1╎    ╎    4     ...ter_functions.jl:42; create_spawn_events!(spawn...
  2╎    ╎     2     @Ba

...

   ╎    ╎    ╎     54    ...\dataframe.jl:245; _preprocess_column(col...
   ╎    ╎    ╎    ╎ 1     @Base\array.jl:352; fill!
   ╎    ╎    ╎    ╎  1     ...tractarray.jl:285; eachindex
   ╎    ╎    ╎    ╎   1     ...ractarray.jl:116; axes1
   ╎    ╎    ╎    ╎    1     ...ractarray.jl:95; axes
  1╎    ╎    ╎    ╎     1     @Base\array.jl:151; size
   ╎    ╎    ╎    ╎ 53    ...\fallbacks.jl:116; allocatecolumn
 53╎    ╎    ╎    ╎  53    @Base\boot.jl:459; Array
 67╎    ╎    ╎    67    ...e\dataframe.jl:226; DataFrame(columns::Vec...
   ╎    ╎    ╎    1     ...e\dataframe.jl:227; DataFrame(columns::Vec...
   ╎    ╎    ╎     1     ...\iterators.jl:167; iterate
   ╎    ╎    ╎    ╎ 1     @Base\array.jl:898; iterate
  1╎    ╎    ╎    ╎  1     @Base\array.jl:924; getindex
  7╎    ╎    ╎    96    ...e\dataframe.jl:229; DataFrame(columns::Vec...
   ╎    ╎    ╎     89    @Base\array.jl:617; convert
   ╎    ╎    ╎    ╎ 89    @Base\array.jl:626; Array
   ╎    ╎    ╎    ╎  6     ...ractarray.jl:1126;

  3╎    ╎    ╎    ╎  26    @Base\array.jl:2664; _grow!
  6╎    ╎    ╎    ╎   6     ...stractset.jl:499; mapfilter(pred::Base...
   ╎    ╎    ╎    ╎   2     ...stractset.jl:500; mapfilter(pred::Base...
   ╎    ╎    ╎    ╎    2     @Base\array.jl:898; iterate
   ╎    ╎    ╎    ╎     2     @Base\array.jl:898; iterate
  1╎    ╎    ╎    ╎    ╎ 1     @Base\array.jl:924; getindex
  1╎    ╎    ╎    ╎    ╎ 1     @Base\array.jl:215; length
  1╎    ╎    ╎    ╎   9     ...stractset.jl:501; mapfilter(pred::Base...
  1╎    ╎    ╎    ╎    1     @Base\array.jl:2649; (::Base.var"#132#13...
   ╎    ╎    ╎    ╎    5     @Base\array.jl:2650; (::Base.var"#132#13...
   ╎    ╎    ╎    ╎     5     @Base\set.jl:66; in
  1╎    ╎    ╎    ╎    ╎ 1     @Base\array.jl:2649; getproperty
   ╎    ╎    ╎    ╎    ╎ 4     @Base\dict.jl:569; haskey
   ╎    ╎    ╎    ╎    ╎  1     @Base\dict.jl:281; ht_keyindex
  1╎    ╎    ╎    ╎    ╎   1     ...e\array.jl:215; length
   ╎    ╎    ╎    ╎    ╎  1     @Base\dict.jl:284; ht_

   ╎    ╎    ╎     1     @Base\array.jl:1108; append!(a::Vector{Flo...
  1╎    ╎    ╎    ╎ 1     @Base\array.jl:1011; _growend!
   ╎    ╎    ╎     7     @Base\array.jl:1109; append!(a::Vector{Str...
   ╎    ╎    ╎    ╎ 7     @Base\array.jl:322; copyto!
   ╎    ╎    ╎    ╎  6     @Base\array.jl:331; _copyto_impl!(dest::V...
  6╎    ╎    ╎    ╎   6     @Base\array.jl:289; unsafe_copyto!
   ╎    ╎    ╎    ╎  1     @Base\array.jl:331; _copyto_impl!
  1╎    ╎    ╎    ╎   1     @Base\array.jl:286; unsafe_copyto!
 11╎    ╎    ╎    11    ...e\insertion.jl:288; _append_or_prepend!(df...
  2╎    ╎    ╎    2     ...e\insertion.jl:289; _append_or_prepend!(df...
   ╎    ╎     12    ...ore_functions.jl:53; update_travel_time!(data_...
   ╎    ╎    ╎ 12    @Base\dict.jl:569; haskey
  1╎    ╎    ╎  1     @Base\dict.jl:0; ht_keyindex(h::Dict{Stri...
  1╎    ╎    ╎  1     @Base\dict.jl:280; ht_keyindex(h::Dict{Stri...
   ╎    ╎    ╎  2     @Base\dict.jl:281; ht_keyindex(h::Dict{Stri...
  2╎    ╎    ╎   

   ╎    ╎    ╎ 1     ...5\src\groups.jl:40; create_group(parent::HDF5...
   ╎    ╎    ╎  1     ...c\properties.jl:882; _link_properties
   ╎    ╎    ╎   1     ...c\properties.jl:20; copy
  1╎    ╎    ╎    1     ...\functions.jl:5864; h5p_copy(plist_id::Int64)
   ╎    ╎    ╎ 1     ...5\src\groups.jl:45; create_group(parent::HDF5...
   ╎    ╎    ╎  1     ...5\src\groups.jl:23; create_group
   ╎    ╎    ╎   1     ...5\src\groups.jl:37; #create_group#19
  1╎    ╎    ╎    1     ...\functions.jl:1770; h5g_create(loc_id::HDF...
   ╎    ╎    130   ...df5_functions.jl:30; store_final_data(data::Dat...
   ╎    ╎     130   ...src\highlevel.jl:96; setindex!
   ╎    ╎    ╎ 130   ...rc\highlevel.jl:113; #setindex!#108
   ╎    ╎    ╎  130   ...src\datasets.jl:305; write
   ╎    ╎    ╎   130   ...rc\datasets.jl:305; #write#57
   ╎    ╎    ╎    130   ...rc\datasets.jl:278; write_dataset
   ╎    ╎    ╎     70    ...c\datasets.jl:281; write_dataset(parent::...
   ╎    ╎    ╎    ╎ 70    ...c\datasets.jl:2

   ╎    ╎    ╎    ╎  4     ...c\datasets.jl:493; write_dataset
   ╎    ╎    ╎    ╎   2     ...efpointer.jl:154; Ref{Cstring}(a::Vect...
  2╎    ╎    ╎    ╎    2     @Base\array.jl:966; setindex!
   ╎    ╎    ╎    ╎   1     ...efpointer.jl:155; Ref{Cstring}(a::Vect...
  1╎    ╎    ╎    ╎    1     ...ssentials.jl:492; setindex!
  1╎    ╎    ╎    ╎   1     ...efpointer.jl:156; Ref{Cstring}(a::Vect...
   ╎    ╎    ╎    ╎  52    ...c\datasets.jl:494; write_dataset
 52╎    ╎    ╎    ╎   52    ...functions.jl:890; h5d_write(dataset_id...
   ╎    ╎    3     ...df5_functions.jl:46; store_final_data(data::Dat...
   ╎    ╎     3     @HDF5\src\groups.jl:39; create_group
   ╎    ╎    ╎ 3     ...5\src\groups.jl:45; create_group(parent::HDF5...
   ╎    ╎    ╎  3     ...5\src\groups.jl:23; create_group
   ╎    ╎    ╎   3     ...5\src\groups.jl:37; #create_group#19
  3╎    ╎    ╎    3     ...\functions.jl:1770; h5g_create(loc_id::HDF...
   ╎    ╎    4     ...df5_functions.jl:47; store_final_data(data::